In [1]:
import pickle
import pandas as pd


In [2]:
#opening our clustered model
with open('finalized_model.sav', 'rb') as fid:
    model = pickle.load(fid)


In [3]:
data = pd.read_csv("Opportunity 2.csv")


In [4]:
data = data.dropna(how='any')


In [5]:
#Converting the whole dataset into smaller case
data=data.apply(lambda x: x.astype(str).str.lower())


In [6]:
comb_frame = data['Discipline'].str.cat(" "+data['Sub tags'])


In [7]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer


In [8]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(comb_frame)


In [9]:
label_dict = {
                 0: "Social Humanities",
                 1: "Management",                  
                 2: "Engineering & Technology  science",
                
            }


In [11]:

# load the complete data in a dataframe
data=pd.read_csv("Opportunity 2.csv")

#Converting the whole dataset into smaller case
data=data.apply(lambda x: x.astype(str).str.lower())

data=data.dropna(how='any')

# create new column in dataframe which is combination of (ID, Sub tags, Discipline) in existing data-frame
data['InputString'] = data['ID'].str.cat(" "+data['Sub tags'].str.cat(" "+data['Discipline']))

data['ClusterPrediction'] = ""


def cluster_predict(str_input):
    Y = vectorizer.transform(list(str_input))
    prediction = model.predict(Y)
    return prediction

# Cluster category for each live course
data['ClusterPrediction']=data.apply(lambda x: cluster_predict(data['InputString']), axis=0)


In [70]:
def recommend_util(str_input):
    
    # match on the basis ID 
    temp_df = data.loc[data['ID'] == str_input]
    temp_df['InputString'] = temp_df['ID'].str.cat(" "+temp_df['Sub tags'].str.cat(" "+temp_df['Discipline']))
    str_input = list(temp_df['InputString'])
    
    
    # Predict category of input string category
    prediction_inp = cluster_predict(str_input)
    prediction_inp = int(prediction_inp)
    # Based on the above prediction 10 random courses are recommended from the whole data-frame
    # Recommendation Logic is kept super-simple for current implementation.
    temp_df = data.loc[data['ClusterPrediction'] == prediction_inp]
    temp_df = temp_df.sample(2)
    
    return list(temp_df['ID'])



In [71]:
if __name__ == '__main__':
    queries = ['1','50']
for query in queries:
    res = recommend_util(query)
    print(res)


['267', '1471']
['127', '1884']


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
data.iloc[[0,266,1470],:]

,ID,Headline,Description,Funding,Deadline,Official Link,Type of opportunity,Sub tags,Discipline,Minimum Qualification / Applicable for,Countries eligible,Location of opportunity,InputString,ClusterPrediction
0,1,international fellowship world forestry center,wfc’s discovery museum was opened in 1971 to e...,nan,10/15/2019,https://www.noticebard.com/call-for-applicatio...,conferences,nan,humanities and social sciences,bachelors,all,usa,1 nan humanities and social sciences,0
266,267,getty/acls postdoctoral fellowships in the his...,acls invites applications for getty/acls postd...,fully funded,10/23/2019,https://ofa.acls.org/,"fellowships, scholarships",nan,humanities and social sciences,doctorate,all,usa,267 nan humanities and social sciences,0
1470,1471,humane studies fellowship 2020-2021,applications are open for the humane studies f...,nan,12/1/2019,https://opportunitydesk.org/2019/11/18/humane-...,fellowships,nan,humanities and social sciences,post doctorate,all,usa,1471 nan humanities and social sciences,0


In [14]:
data.iloc[[49,126,1883]]

,ID,Headline,Description,Funding,Deadline,Official Link,Type of opportunity,Sub tags,Discipline,Minimum Qualification / Applicable for,Countries eligible,Location of opportunity,InputString,ClusterPrediction
49,50,kectil leadership program 2020 for young leade...,applications are invited for the kectil web-ba...,partially funded,10/31/2019,https://kectil.com/holding-page-123/,fellowships,nan,all,all,all,online,50 nan all,1
126,127,aspen institute’s new voices fellowship 2020 f...,applications are invited for the aspen institu...,fully funded,10/15/2019,http://newvoicesfellows.aspeninstitute.org/nom...,fellowships,nan,all,all,all,usa,127 nan all,1
1883,1884,amgen scholars asia program 2020,amgen scholars is hosted at four premier educa...,nan,2/3/2020,http://amgenscholars.com/asia-program/,fellowships,nan,all,bachelors,all,none,1884 nan all,1


For <b>Query 1-'1'</b>: It recommended us ['267', '1471'] this array.


<br>
<b>1</b>	-International Fellowship World Forestry Center
<br><b>Deadline:</b>	10/15/2019	<b>Link:</b>https://www.noticebard.com/call-for-applications-international-fellowship-world-forestry-center/  <b>Type of Opportunity:</b>Conferences		<b>Discipline:</b>Humanities and Social Sciences	<b>Minimum Qualification / Applicable for:</b>Bachelors	<b>Countries Eligible:</b>ALL	<b>Location of opportunity:</b>USA
		

    
<br>
<br>
<i>The recommended options are:</i>

<b>267</b>	-Getty/ACLS Postdoctoral Fellowships in the History of Art		<br><b>Funded:</b>Fully funded	<b>Deadline:</b>10/23/2019	<b>Link:</b>https://ofa.acls.org/	<b>Type of Opportunity:</b>Fellowships, Scholarships		<b>Discipline:</b>Humanities and Social Sciences	<b>Minimum Qualification / Applicable for:</b>Doctorate	   <b>Countries Eligible:</b>All <b>Location of opportunity:</b>	USA

<br>
<br>
<b>1471</b>	-Humane Studies Fellowship 2020-2021		<br>	<b>Deadline:</b>12/1/2019	<b>Link:</b>https://opportunitydesk.org/2019/11/18/humane-studies-fellowship-2020-2021/	<b>Type of Opportunity:</b>Fellowships		<b>Discipline:</b>Humanities and Social Sciences	<b>Minimum Qualification / Applicable for:</b>Post Doctorate	   <b>Countries Eligible:</b>All <b>Location of opportunity:</b>	USA
<br>




For <b>Query 2-'50'</b>: It recommended us ['127', '1884'] this array.


<br>
<b>50</b>	-Kectil Leadership Program 2020 for Young Leaders in Developing Countries
<br><b>Funded:</b>Partially Funded	 <b>Deadline:</b>	10/31/2019	<b>Link:</b>https://kectil.com/holding-page-123/  <b>Type of Opportunity:</b>Fellowships			<b>Discipline:</b>All	<b>Minimum Qualification / Applicable for:</b>All	<b>Countries Eligible:</b>ALL	<b>Location of opportunity:</b>Online
		

<br>
<br>
<i>The recommended options are:</i>

<b>127	</b>	-Aspen Instituteâ€™s New Voices Fellowship 2020 for Development Professionals	<br><b>Funded:</b>Fully funded	<b>Deadline:</b>10/15/2019<b>Link:</b>http://newvoicesfellows.aspeninstitute.org/Nominations<b>Type of Opportunity:</b>Fellowships			<b>Discipline:</b>All	<b>Minimum Qualification / Applicable for:</b>All	   <b>Countries Eligible:</b>All <b>Location of opportunity:</b>	USA


<br>
<br>
<b>1884</b>	-Amgen Scholars Asia Program 2020		<br>	<b>Deadline:</b>	2/3/2020	<b>Link:</b>http://amgenscholars.com/asia-program/	<b>Type of Opportunity:</b>Fellowships		<b>Discipline:</b>All	<b>Minimum Qualification / Applicable for:</b>	Bachelors	   <b>Countries Eligible:</b>All <b>Location of opportunity:</b>	None
<br>

	
